# 2 The Standard Exception Hierarchy

In [1]:
s = [1, 4, 6]

In [3]:
# s[5]

# ---------------------------------------------------------------------------
# IndexError                                Traceback (most recent call last)
# <ipython-input-2-b5dece75d686> in <module>
# ----> 1 s[5]

# IndexError: list index out of range

In [4]:
d = dict(a=65, b=66, c=67)

In [6]:
# d['x']

# ---------------------------------------------------------------------------
# KeyError                                  Traceback (most recent call last)
# <ipython-input-5-7657742692bd> in <module>
# ----> 1 d['x']

# KeyError: 'x'

In [7]:
IndexError.mro()

[IndexError, LookupError, Exception, BaseException, object]

In [8]:
KeyError.mro()

[KeyError, LookupError, Exception, BaseException, object]

In [9]:
def lookups():
    s = [1, 4, 6]
    try:
        item = s[5]
    except LookupError:
        print("Handled IndexError")

    d = dict(a=65, b=66, c=67)
    try:
        value = d['x']
    except LookupError:
        print("Handled KeyError")

In [10]:
lookups()

Handled IndexError
Handled KeyError


# 3 Exception Payloads

In [11]:
def median(iterable):
    """Obtain the central value of a series.

    Sorts the iterable and returns the middle value if there is an even
    number of elements, or the arithmetic mean of the middle two elements
    if there is an even number of elements.

    Args:
        iterable: A series of orderable items.

    Returns:
        The median value.
    """
    items = sorted(iterable)
    if len(items) == 0:
        raise ValueError("median() arg is an empty sequence")
    median_index = (len(items) - 1) // 2
    if len(items) % 2 != 0:
        return items[median_index]
    return (items[median_index] + items[median_index + 1]) / 2.0

In [14]:
try:
    median([])
except ValueError as e:
    print("Payload:", e.args)
    print(str(e))

Payload: ('median() arg is an empty sequence',)
median() arg is an empty sequence


In [15]:
try:
    b'\x81'.decode('utf-8')
except UnicodeError as e:
    print(e)
    print("encoding:", e.encoding)
    print("reason:", e.reason)
    print("object:", e.object)
    print("start:", e.start)
    print("end", e.end)

'utf-8' codec can't decode byte 0x81 in position 0: invalid start byte
encoding: utf-8
reason: invalid start byte
object: b'\x81'
start: 0
end 1


# 4 Defining New Exceptions

In [16]:
import math

In [17]:
def triangle_area(a, b, c):
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return a

In [19]:
triangle_area(3, 4, 5)

6.0

In [21]:
# triangle_area(3, 4, 10)

# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-20-5ab8071fe29d> in <module>
# ----> 1 triangle_area(3, 4, 10)

# <ipython-input-17-87a96d8a71d4> in triangle_area(a, b, c)
#       1 def triangle_area(a, b, c):
#       2     p = (a + b + c) / 2
# ----> 3     a = math.sqrt(p * (p - a) * (p - b) * (p - c))
#       4     return a

# ValueError: math domain error

In [22]:
class TriangleError(Exception):
    pass

In [23]:
def triangle_area(a, b, c):
    sides = sorted((a, b, c))
    if sides[2] > sides[0] + sides[1]:
        raise TriangleError("Illegal triangle", sides)
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return a

In [25]:
# triangle_area(3, 4, 10)

# ---------------------------------------------------------------------------
# TriangleError                             Traceback (most recent call last)
# <ipython-input-24-5ab8071fe29d> in <module>
# ----> 1 triangle_area(3, 4, 10)

# <ipython-input-23-514812d890a6> in triangle_area(a, b, c)
#       2     sides = sorted((a, b, c))
#       3     if sides[2] > sides[0] + sides[1]:
# ----> 4         raise TriangleError("Illegal triangle", sides)
#       5     p = (a + b + c) / 2
#       6     a = math.sqrt(p * (p - a) * (p - b) * (p - c))

# TriangleError: ('Illegal triangle', [3, 4, 10])

In [26]:
class TriangleError(Exception):

    def __init__(self, text, sides):
        super().__init__(text)
        self._sides = tuple(sides)

    @property
    def sides(self):
        return self._sides

    def __str__(self):
        return "'{}' for sides {}".format(self.args[0], self._sides)

    def __repr__(self):
        return "TriangleError({!r}, {!r}".format(self.args[0], self._sides)

In [28]:
# triangle_area(3, 4, 10)

# ---------------------------------------------------------------------------
# TriangleError                             Traceback (most recent call last)
# <ipython-input-27-5ab8071fe29d> in <module>
# ----> 1 triangle_area(3, 4, 10)

# <ipython-input-23-514812d890a6> in triangle_area(a, b, c)
#       2     sides = sorted((a, b, c))
#       3     if sides[2] > sides[0] + sides[1]:
# ----> 4         raise TriangleError("Illegal triangle", sides)
#       5     p = (a + b + c) / 2
#       6     a = math.sqrt(p * (p - a) * (p - b) * (p - c))

# TriangleError: 'Illegal triangle' for sides (3, 4, 10)

# 5 Chaining Exceptions

In [29]:
import sys

In [31]:
# try:
#     a = triangle_area(3, 4, 10)
#     print(a)
# except TriangleError as e:
#     print(e, file=sys.stdin)
    
# ---------------------------------------------------------------------------
# TriangleError                             Traceback (most recent call last)
# <ipython-input-30-a762e56fc918> in <module>
#       1 try:
# ----> 2     a = triangle_area(3, 4, 10)
#       3     print(a)

# <ipython-input-23-514812d890a6> in triangle_area(a, b, c)
#       3     if sides[2] > sides[0] + sides[1]:
# ----> 4         raise TriangleError("Illegal triangle", sides)
#       5     p = (a + b + c) / 2

# TriangleError: 'Illegal triangle' for sides (3, 4, 10)

# During handling of the above exception, another exception occurred:

# UnsupportedOperation                      Traceback (most recent call last)
# <ipython-input-30-a762e56fc918> in <module>
#       3     print(a)
#       4 except TriangleError as e:
# ----> 5     print(e, file=sys.stdin)

# UnsupportedOperation: not writable

In [32]:
import io

In [33]:
try:
    a = triangle_area(3, 4, 10)
    print(a)
except TriangleError as e:
    try:
        print(e, file=sys.stdin)
    except io.UnsupportedOperation as f:
        print(e)
        print(f)
        print(f.__context__ is e)

'Illegal triangle' for sides (3, 4, 10)
not writable
True


In [34]:
def inclination(dx, dy):
    return math.degrees(math.atan(dy / dx))

In [35]:
inclination(3, 5)

59.03624346792648

In [37]:
# inclination(0, 5)

# ---------------------------------------------------------------------------
# ZeroDivisionError                         Traceback (most recent call last)
# <ipython-input-36-393cbe0c53e7> in <module>
# ----> 1 inclination(0, 5)

# <ipython-input-34-e3071db817c5> in inclination(dx, dy)
#       1 def inclination(dx, dy):
# ----> 2     return math.degrees(math.atan(dy / dx))

# ZeroDivisionError: division by zero

In [38]:
class InclinationError(Exception):
    pass

In [39]:
def inclination(dx, dy):
    try:
        return math.degrees(math.atan(dy / dx))
    except ZeroDivisionError as e:
        raise InclinationError("Slope cannot be vertical") from e

In [40]:
inclination(0, 5)

InclinationError: Slope cannot be vertical

In [41]:
try:
    inclination(0, 5)
except InclinationError as e:
    print(e)
    print(e.__cause__)

Slope cannot be vertical
division by zero


# 6 Traceback Objects

In [43]:
import traceback

In [45]:
try:
    inclination(0, 5)
except InclinationError as e:
    print(e.__traceback__)
    traceback.print_tb(e.__traceback__)
    s = traceback.format_tb(e.__traceback__)
    print(s)
    
print("Finished")

['  File "<ipython-input-45-02689c8f6b10>", line 2, in <module>\n    inclination(0, 5)\n', '  File "<ipython-input-39-782e0595348f>", line 5, in inclination\n    raise InclinationError("Slope cannot be vertical") from e\n']
Finished


  File "<ipython-input-45-02689c8f6b10>", line 2, in <module>
    inclination(0, 5)
  File "<ipython-input-39-782e0595348f>", line 5, in inclination
    raise InclinationError("Slope cannot be vertical") from e


# 7 Assertions Internal Invariants

In [47]:
# assert False, "The condition was false"

# ---------------------------------------------------------------------------
# AssertionError                            Traceback (most recent call last)
# <ipython-input-46-74bb2f6f24c8> in <module>
# ----> 1 assert False, "The condition was false"

# AssertionError: The condition was false

In [48]:
assert 5 > 2, "You are in a defective universe!"

In [49]:
def modulus_three(n):
    r = n % 3
    if r == 0:
        print("Multiple of 3")
    elif r == 1:
        print("Remainder 1")
    else:
        assert r == 2, "Remainder is not 2"
        print("Remainder 2")

In [50]:
def modulus_four(n):
    r = n % 4
    if r == 0:
        print("Multiple of 4")
    elif r == 1:
        print("Remainder 1")
    else:
        assert r == 2, "Remainder is not 2"
        print("Remainder 2")

In [51]:
modulus_four(4)

Multiple of 4


In [53]:
# modulus_four(3)

# ---------------------------------------------------------------------------
# AssertionError                            Traceback (most recent call last)
# <ipython-input-52-622201bfce93> in <module>
# ----> 1 modulus_four(3)

# <ipython-input-50-929d9eb09a2f> in modulus_four(n)
#       6         print("Remainder 1")
#       7     else:
# ----> 8         assert r == 2, "Remainder is not 2"
#       9         print("Remainder 2")

# AssertionError: Remainder is not 2

In [54]:
def modulus_four(n):
    r = n % 4
    if r == 0:
        print("Multiple of 4")
    elif r == 1:
        print("Remainder 1")
    elif r == 2:
        print("Remainder 2")
    else:
        assert r == 3, "Remainder is not 3"
        print("Remainder 3")

In [55]:
modulus_four(3)

Remainder 3


In [56]:
def modulus_four(n):
    r = n % 4
    if r == 0:
        print("Multiple of 4")
    elif r == 1:
        print("Remainder 1")
    elif r == 2:
        print("Remainder 2")
    elif r == 3:
        print("Remainder 3")
    else:
        assert False, "This should never happen"